In [10]:
import sifce
import pandas as pd
import numpy as np
from bilby.core.prior import Uniform, Sine, Cosine, PowerLaw, Constraint, DeltaFunction
from pycbc.detector import Detector
from bilby.core.prior import PriorDict
from pycbc.psd import analytical

In [2]:
# get ifos
ifos=['H1', 'L1', 'V1']
ifos_det = []

for ifo in ifos:
    ifos_det.append(Detector(ifo))

In [50]:
prior_gw150914 = dict(
                    mass_1= PowerLaw(alpha=-1, name='mass_1', minimum=10, maximum=80),
                    mass_2= PowerLaw(alpha=-1, name='mass_2', minimum=10, maximum=80),
                    a_1 = Uniform(name='a_1', minimum=0, maximum=0.99),
                    a_2 = Uniform(name='a_2', minimum=0, maximum=0.99),
                    tilt_1 = Sine(name='tilt_1'),
                    tilt_2 = Sine(name='tilt_2'),
                    phi_12 = Uniform(name='phi_12', minimum=0, maximum=2 * np.pi, boundary='periodic'),
                    phi_jl = Uniform(name='phi_jl', minimum=0, maximum=2 * np.pi, boundary='periodic'),
                    theta_jn =  Sine(name='theta_jn'),
                    phase =  Uniform(name='phase', minimum=0, maximum=2 * np.pi, boundary='periodic'),
                    reference_frequency = DeltaFunction(20),
                    dec=Cosine(name='dec'),
                    ra=Uniform(name='ra', minimum=0, maximum=2 * np.pi, boundary='periodic'),
                    psi =  Uniform(name='psi', minimum=0, maximum=np.pi, boundary='periodic'),)

In [51]:
from sifce import population
from sifce import datatools

In [52]:
prior = PriorDict(dictionary = prior_gw150914)

pop = population.SimulationSet(distribution=prior)
pop.sample_distribution(10)
# print(pop.simulations_dataframe)

In [53]:
sample_rate=1024
flow = 20.0
delta_f = 1.0/8
flen = int(sample_rate/delta_f)
psd = analytical.aLIGOZeroDetHighPower(flen, delta_f, flow)

psd_dict = {'H1': psd, 'L1': psd, 'V1':psd}
apx='IMRPhenomXPHM'

In [67]:
# this loop is not efficient!

temp_pd = pop.simulations_dataframe
opt_snr_lst = list()

for index, row in temp_pd.iterrows():
    tmp_dict = row.to_dict()
    
    # get hp and hc:
    param_args = { k: tmp_dict[k] for k in pop.waveform_generation_labels}
    strain = dict()
    strain['plus'], strain['cross'] = datatools.compute_hphc_fd(param_args, apx)
    
    # projection into antenna:
    position_args = { k: tmp_dict[k] for k in pop.detector_position_labels}
    projected_strains = dict()
    for ifo_name, ifo_detec in zip(ifos, ifos_det):
        projected_strains[ifo_name] = dict(template = datatools.project_and_combine(position_args, strain, 0, detector_obj=ifo_detec))

    # calculate snr
    opt_snr_dict, temp_snr_dict = datatools.compute_snr_fd(projected_strains, psd_dict)
    opt_snr_lst.append(opt_snr_dict['net'])

pop.simulations_dataframe['opt_snr'] = opt_snr_lst

In [66]:
pop.simulations_dataframe

,mass_1,mass_2,spin_1x,spin_1y,spin_1z,spin_2x,spin_2y,spin_2z,theta_jn,phase,...,tilt_2,phi_12,phi_jl,reference_frequency,cal_idx,opt_snr,template_snr,iota,phi_1,phi_2
0,24.369228,21.077266,0.151356,0.092244,-0.307855,0.450266,0.049900,0.076554,1.253422,1.710571,...,1.403393,5.846221,5.125727,20.0,NaN,16287.902368,NaN,1.379622,0.547338,0.110374
1,10.882155,54.399002,0.137527,-0.473340,0.480967,-0.738539,-0.032743,0.601190,1.615053,3.019187,...,0.888042,4.473933,3.105789,20.0,NaN,17235.753521,NaN,1.593921,4.995150,3.185898
2,13.350293,69.753503,0.368885,0.107379,-0.273710,0.036752,0.093210,-0.009877,1.562078,1.251053,...,1.669054,0.911952,5.481084,20.0,NaN,14835.367220,NaN,1.671112,0.283264,1.195216
3,10.987617,42.040148,-0.051570,-0.063687,-0.069562,-0.397285,0.002821,0.424580,1.211676,2.542757,...,0.752212,5.385946,2.451801,20.0,NaN,9837.677701,NaN,1.059357,4.031731,3.134491
4,36.533186,63.419351,-0.019265,0.048438,0.120930,-0.279345,0.309791,0.276123,0.844206,0.080713,...,0.986069,0.355221,5.618330,20.0,NaN,44721.026861,NaN,0.960609,1.949340,2.304561
5,12.268306,57.860597,-0.004480,-0.071664,-0.135400,-0.091959,-0.060886,0.265581,2.358851,6.123353,...,0.393604,5.359660,0.358227,20.0,NaN,24242.021156,NaN,2.319236,4.649953,3.726428
6,19.260571,18.776120,0.004973,-0.022116,0.032299,0.097976,0.067327,-0.008514,3.027997,2.872709,...,1.642295,1.951683,6.217293,20.0,NaN,24592.247853,NaN,3.027206,4.933571,0.602068
7,12.917813,24.385791,0.237126,0.796783,0.484017,-0.029793,0.333197,-0.160719,1.238856,6.125382,...,2.018672,0.378435,4.477212,20.0,NaN,13641.049281,NaN,1.452496,1.281538,1.659974
8,35.892288,60.229804,0.151442,0.180741,0.263375,-0.230674,0.306278,-0.424096,0.293049,4.968248,...,2.406514,1.342942,3.274046,20.0,NaN,50305.037444,NaN,0.384185,0.873370,2.216313
9,20.961997,19.326058,-0.091461,-0.001173,0.047456,-0.304223,-0.325674,-0.540660,2.235985,2.409728,...,2.452214,0.806618,3.170517,20.0,NaN,24909.177058,NaN,2.234166,3.154414,3.961033
